In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

%reload_ext autoreload
%autoreload 2

!wget http://23.105.212.181:8000/daf.tar.gz -O daf.tar.gz 
!tar xvf daf.tar.gz

In [ ]:

%reload_ext autoreload
%autoreload 2

In [ ]:
def download_file_one_at_a_time(file_name, directory=".", overwrite=True):
    if overwrite:
        run_process_print("wget http://107.182.177.9:8000/{0} -O \"{1}/{0}\"".format(file_name, directory))
    else:
        run_process_print("[ -f {1}/{0} ] || wget http://23.105.212.181:8000/{0} -P {1}".format(file_name, directory))


def run_process_print(command_str):
    print(command_str.split())
    get_ipython().system(command_str)


def setup_gdrive():
    download_file_one_at_a_time("gdrive")
    s = """chmod +x ./gdrive
    mkdir $HOME/.gdrive 
    chmod +x ./gdrive
    """

    [(print(c.strip()), run_process_print(c.strip())) for c in s.split('\n')]
    # download_file_one_at_a_time("token_v2.json", "$HOME/.gdrive")
    str = """{
        "access_token": "ya29.GlsWB6DpEzK1qbegW-7FGy84GUtdR8O57aoq3i73DiFLlwpGxG1hZGwCVLiBIFNCDIw0zgQ6Fs4aBkf1YWbc30_yJMLCtv1E1b20nqMF2gRF3cJU_Ks-xnsaF5WV",
        "token_type": "Bearer",
        "refresh_token": "1/uxgj61NZOFM_LkIZd6QHpGX0Nj8bm9004DK68Ywu0pU",
        "expiry": "2019-05-27T06:11:29.604819094-04:00"
    }"""
    with open("token_v2.json", 'wb') as f:
        f.write(bytes(str, 'utf-8'))
    run_process_print('mv token_v2.json $HOME/.gdrive')

In [ ]:
download_file_one_at_a_time('Train.py')
%run Train.py

In [ ]:
setup_gdrive()

In [ ]:
# !./gdrive download 1meRU32M5cm0RXDI2_GaeDCHaGvISDCZE

In [ ]:
!./gdrive list

In [ ]:
!./gdrive download 10_bzbaYhth9wurN9FzCUxCZ-rsiWHC3l

In [ ]:
!mkdir models ; mv *pth models

In [ ]:
!wget https://www.kaggleusercontent.com/kf/17274750/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..iW7qeXH6bvVzU4hfnmXOBA.OnKmLhEQZiG7BQC0speCnPF-mbQc0iuxXpc-gEmSJ-X9WKniYzV1X9pSKvO1EM9LhCrCwGY6RuBUdlIfZ7xk39eQlHWUlpfcPV7c_30clh-FAbgxuohA8a7ld1r3bj1N.PIYIts3bSaPNlNf2XrRkAA/models/dr-stage1_2.pth

In [ ]:
learner.load('dr-stage1_2')
k.learn.model.eval()

In [ ]:
k.learn.data

In [ ]:
k.learn = learner

In [ ]:
kf = KFold(n_splits=k.nfolds, shuffle=True, random_state=k.SEED)
valid_idx = list(kf.split(list(range(len(Path(k.TRAIN).ls())))))[0][1]
k.learn.data= (SegmentationItemList.from_folder(k.TRAIN)
                .split_by_idx(valid_idx)
                .label_from_func(lambda x: str(x).replace('train', 'masks'), classes=[0, 1])
                .add_test(Path(k.TEST).ls(), label=None)
                .databunch(path=Path('.'), bs=k.bs)
                .normalize(k.stats))

In [ ]:
import pdb

In [ ]:
k.learn.data

In [ ]:
k.learn.model.eval()

In [ ]:
output_WIP=k.learn.get_preds(DatasetType.Test, with_loss=False)

In [ ]:
    def predict_on_test_post_process(self, output, thr=None):
        # ### Submission
        # convert predictions to byte type and save
        output_prob = torch.sigmoid(output)

        preds_save = (output_prob * 255.0).byte()
        torch.save(preds_save, 'preds_test.pt')

        output_WIP = output_prob

        # Generate rle encodings (images are first converted to the original size)
        thr = self.best_thr if thr is None else thr
        output_WIP = (output_WIP > thr).int()

        # If any pixels are predicted for an empty mask, the corresponding image gets zero score during evaluation. While prediction of no pixels for an empty mask gives perfect score. Because of this penalty it is resonable to set masks to zero if the number of predicted pixels is small. This trick was quite efective in [Airbus Ship Detection Challenge](https://www.kaggle.com/iafoss/unet34-submission-tta-0-699-new-public-lb).
        output_WIP[output_WIP.view(output_WIP.shape[0], -1).sum(-1) < self.noise_th, ...] = 0.0

        #output_WIP = output_WIP.numpy()

        rles = []
        rlesNoT = []
        t = torchvision.transforms.ToPILImage()
        for p in progress_bar(output_WIP):
            im = t(p)
            #p_1_channel=(p.T * 255).astype(np.uint8)
            im = im.resize((1024, 1024))
            im = np.asarray(im)
            if debug_trace: set_trace()
            im = im*255  # the mask2rle will use this

            im_4_rle = np.copy(np.transpose(im))
            if debug_trace: set_trace()
            rles.append(mask2rle(im_4_rle, 1024, 1024))
            rlesNoT.append(mask2rle(im, 1024, 1024))
            del im
            del im_4_rle


        ids = [o.stem for o in self.learn.data.test_ds.items]
        sub_df = pd.DataFrame({'ImageId': ids, 'EncodedPixels': rles})
        sub_df.loc[sub_df.EncodedPixels == '', 'EncodedPixels'] = '-1'
        sub_df.to_csv('submission_.csv', index=False)
        
        sub_df_noT = pd.DataFrame({'ImageId': ids, 'EncodedPixels': rlesNoT})
        sub_df_noT.loc[sub_df.EncodedPixels == '', 'EncodedPixels'] = '-1'
        sub_df_noT.to_csv('submission_not_T_.csv', index=False)
        
        print(sub_df.head())

In [ ]:
k.preds_test.shape

In [ ]:
# %load ../input/siim-acr-pneumothorax-segmentation/mask_functions.py
import numpy as np

def mask2rle(img, width, height):
    rle = []
    lastColor = 0;
    currentPixel = 0;
    runStart = -1;
    runLength = 0;

    for x in range(width):
        for y in range(height):
            currentColor = img[x][y]
            if currentColor != lastColor:
                if currentColor == 255:
                    runStart = currentPixel;
                    runLength = 1;
                else:
                    rle.append(str(runStart));
                    rle.append(str(runLength));
                    runStart = -1;
                    runLength = 0;
                    currentPixel = 0;
            elif runStart > -1:
                runLength += 1
            lastColor = currentColor;
            currentPixel+=1;

    return " ".join(rle)

def rle2mask(rle, width, height):
    mask= np.zeros(width* height)
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        current_position += start
        mask[current_position:current_position+lengths[index]] = 255
        current_position += lengths[index]

    return mask.reshape(width, height)


In [ ]:
ls ../input/siim-acr-pneumothorax-segmentation


In [ ]:
sys.path.insert(0, '../input/siim-acr-pneumothorax-segmentation')

In [ ]:
debug_trace=False

In [ ]:
!head -n 3  submissio*.csv

In [ ]:
predict_on_test_post_process(k, output_WIP[0], thr=0.9)

In [ ]:
!head -n 3 submission_not_T.csv

In [ ]:
!head b

In [ ]:
!diff submission.csv submission_not_T.csv

In [ ]:
!./gdrive upload submission_not_T_.csv

In [ ]:
pdb.pm()

In [ ]:
ls

In [ ]:
ddd=torch.load('preds_test.pt')

In [ ]:
ddd

In [ ]:
preds_on_test, _ = k.learn.get_preds(DatasetType.Test, with_loss=False)

In [ ]:
preds_on_test.min()

In [ ]:
!find ../input -name "util*" 

In [ ]:
ls -lh

In [ ]:
k.data0=k.learn.data

In [ ]:
predict_on_test_post_process(k, preds_on_test, 0.7)

In [ ]:
pdb.pm()

In [ ]:
!pip install kaggle 
!mkdir $HOME/.kaggle 
!echo '{"username":"k1gaggle","key":"f51513f40920d492e9d81bc670b25fa9"}' > $HOME/.kaggle/kaggle.json
!chmod 600 $HOME/.kaggle/kaggle.json

In [ ]:
!kaggle competitions submit  -f submission_not_T_.csv -m 'daf_no_tta_0.9_th_first_100_noiseth_no_T' siim-acr-pneumothorax-segmentation

In [ ]:
!kaggle competitions submissions siim-acr-pneumothorax-segmentation

In [ ]:
8006/2669

In [ ]:
preds, ys = k.learn.get_preds(DatasetType.Valid)

In [ ]:
preds.shape

In [ ]:
preds.max()

In [ ]:
data.valid_ds[3][0].data

In [ ]:
outprobs = torch.sigmoid(preds)

In [ ]:
ys.shape

In [ ]:
data

In [ ]:
!head -n 6 submission_not_T_.csv submission_.csv

In [ ]:
!gdrive upload submission_not_T_.csv

In [ ]:
ys.shape

In [ ]:
ys.sum((2,3)).sort(descending=True,dim=0).indices

In [ ]:
kdata.valid_ds[987][0]

In [ ]:
plot_idx[1]

In [ ]:
kdata.valid_ds[987][1].data.dtype

In [ ]:
ys[987].data.numpy().transpose(1,2,0).shape

In [ ]:
outprobs_h = (outprobs > 0.9)

In [ ]:
outprobs_h.max()

In [ ]:
rows_start = 1500
rows_end = 1520
plot_idx = ys.sum((2,3)).sort(descending=True, dim=0).indices[rows_start:rows_end]
#print(plot_idx)
for idx in plot_idx:
    idd = idx.data.numpy()[0]
    fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(12, 4))
    ax0.imshow(kdata.valid_ds[idd][0].data.numpy().transpose(1,2,0))
    ax1.imshow(ys[idd].data.numpy().transpose(1,2,0).reshape(256,256), vmin=0, vmax=1)
    ax2.imshow(outprobs_h[idd].data.numpy().transpose(1,2,0).reshape(256,256), vmin=0, vmax=1)
    ax1.set_title('Targets')
    ax2.set_title('Predictions')

In [ ]:
pdb.pm()